In [ ]:
%pip install databricks-genai-inference scrapegraphai pandas nest_asyncio playwright
dbutils.library.restartPython()

In [ ]:
!playwright install
!playwright install-deps

In [ ]:
# 1. Define the schema
schema = "hackathon_schema"

# 2. Create the schema
create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {schema}"
spark.sql(create_schema_query)

# 3. Define the table name
table_name = "users"

# 4. Create the table with the specified columns
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.{table_name} (
    user_id INT,
    username STRING,
    password STRING,
    watchlist STRING
)
"""

# Execute the query
spark.sql(create_table_query)

# 5. Verify the table
verify_query = f"DESCRIBE TABLE {schema}.{table_name}"
result = spark.sql(verify_query)
result.show()


+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|  user_id|      int|   NULL|
| username|   string|   NULL|
| password|   string|   NULL|
|watchlist|   string|   NULL|
+---------+---------+-------+



In [ ]:
# Sample data to insert
# user_id = 1
# username = "elon"
# password = "admin123"
# watchlist = "Tesla"

# # Insert the data
# insert_query = f"""
# INSERT INTO hackathon_schema.users (user_id, username, password, watchlist)
# VALUES ({user_id}, '{username}', '{password}', '{watchlist}')
# """
# spark.sql(insert_query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [ ]:
# Verify the insertion
result = spark.sql("SELECT * FROM hackathon_schema.users")
result.show()

+-------+--------+--------+---------+
|user_id|username|password|watchlist|
+-------+--------+--------+---------+
|      1|    elon|admin123|    Tesla|
+-------+--------+--------+---------+



Get news articles from DuckDuckGo

In [ ]:
query = """
SELECT watchlist
FROM hackathon_schema.users
Limit 1
"""
result = spark.sql(query).collect()

company = result[0]['watchlist']
print(company)

Tesla


In [ ]:
import requests

url = "https://duckduckgo10.p.rapidapi.com/search/news"

# call API to get the news on a company from the watchlist of users table
querystring = {"safeSearch":"off","term":company.lower(),"region":"us-en","time":"m"}

duckduckapi_key = "YOUR API"

headers = {
	"X-RapidAPI-Key": duckduckapi_key,
	"X-RapidAPI-Host": "duckduckgo10.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
response = response.json()

print(response)

{'status': 200, 'message': 'OK', 'data': [{'date': 1714951037, 'excerpt': '<b>Tesla\'s</b> Cybertruck doesn\'t appear as "apocalypse-proof" as suggested, with the $60,000+ vehicle getting stuck in rough terrain and facing recalls.', 'image': 'https://i.insider.com/663527e00dfb1341e9027b5b?width=1200&format=jpeg', 'relativeTime': '10 hours ago', 'syndicate': 'Bing', 'title': "Tesla's 'apocalypse-proof' Cybertruck thwarted by sand, snow, and regulator recalls", 'url': 'https://www.msn.com/en-us/news/technology/teslas-apocalypse-proof-cybertruck-thwarted-by-sand-snow-and-regulator-recalls/ar-BB1lS6Ig', 'isOld': False}, {'date': 1714912200, 'excerpt': 'As Elon Musk ditches plans for a $25,000 <b>Tesla</b>, meet a truly affordable EV: the Zhidou Rainbow. It costs less than an ebike and is capable of completing the average US daily commute with charge to spare.', 'image': 'https://media.wired.com/photos/6635676744feaf4b1663c08a/191:100/w_1280,c_limit/Zhidou-Rainbow-2.jpg', 'relativeTime': '2

Scrape articles from URLs using ScrapeGraphAI

In [ ]:
# check how many articles we have
articles = response['data']
print(len(articles))

30


In [ ]:
import os
from scrapegraphai.graphs import SmartScraperGraph
import pandas as pd
import json
import nest_asyncio
import asyncio
import playwright
from json import JSONDecodeError

async def run_scrape(url, graph_config):
    smart_scraper_graph = SmartScraperGraph(
        prompt="Extract article text",
        source=url,
        config=graph_config
        )

    extracted_dict = smart_scraper_graph.run()
    return extracted_dict

nest_asyncio.apply()

openai_key = "YOUR API"

graph_config = {
    "llm": {
        "api_key": openai_key,
        "model": "gpt-3.5-turbo",
    },
}

articles = response['data']

for article in articles:
    url = article['url']

    # does not scrape well from these websites
    if "www.msn.com" not in url and "thedailybeast.com" not in url:
        extracted_dict = asyncio.run(run_scrape(url, graph_config))
        article['article_text'] = next(iter(extracted_dict.values()))
    else:
        article['article_text'] = "No article text found"

# Convert to JSON string
articles_json = json.dumps(articles)

# Convert to DataFrame
df_raw = pd.read_json(articles_json)
print(df_raw.head(10))


/local_disk0/.ephemeral_nfs/envs/pythonEnv-410bb54c-5726-4437-bfc8-18159466a7c1/lib/python3.10/site-packages/scrapegraphai/utils/remover.py:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')
/local_disk0/.ephemeral_nfs/envs/pythonEnv-410bb54c-5726-4437-bfc8-18159466a7c1/lib/python3.10/site-packages/scrapegraphai/utils/remover.py:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')
/local_disk0/.ephemeral_nfs/envs/pythonEnv-410bb54c-5726-4437-bfc8-18159466a7c1/lib/python3.10/site-packages/scrapegraphai/utils/remover.py:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful

                 date  ...                                       article_text
0 2024-05-05 23:17:17  ...                              No article text found
1 2024-05-05 12:30:00  ...  As Elon Musk steps away, yet again, from the i...
2 2024-05-05 18:42:00  ...                              No body content found
3 2024-05-05 09:00:00  ...                              No article text found
4 2024-05-05 15:13:00  ...                              No article text found
5 2024-05-05 20:18:00  ...                              No article text found
6 2024-05-05 19:00:00  ...                              No body content found
7 2024-05-05 00:38:16  ...                              No article text found
8 2024-05-04 19:00:28  ...                              No article text found
9 2024-05-03 15:34:00  ...                              No article text found

[10 rows x 9 columns]


Summarize long articles

In [ ]:
# check how many articles above 1000 characters 
long_articles = df_raw[df_raw["article_text"].str.len() > 1000]
print(len(long_articles))

3


In [ ]:
# from langchain_core.prompts import PromptTemplate
# from langchain.chat_models import ChatDatabricks

# model = ChatDatabricks(endpoint="databricks-dbrx-instruct")

# for index, row in df_raw.iterrows():
#     # if article is above 1000 characters then summarize it
#     if len(row['article_text']) > 1000:
#         query = row['article_text']
#         prompt = PromptTemplate(
#             template="Summarize this news article keeping the most important details.\n{format_instructions}\n{query}\n",
#             input_variables=["query", "text"]
#         )

#         chain = prompt | model
#         response = chain.invoke({"format_instructions": None, "query": query})
#         df_raw.at[index, 'summary'] = response.content
#     # else keep the article text as is
#     else:
#         df_raw.at[index, 'summary'] = row['article_text']

# print(df_raw['summary'].head(5))

0                                No body content found
1    The article discusses the Zhidou Rainbow, a co...
2                                No article text found
3                                No article text found
4                                No article text found
Name: summary, dtype: object


In [ ]:
# # check how many articles above 1000 characters 
# long_articles = df_raw[df_raw["summary"].str.len() > 1000]
# print(len(long_articles))

0


Data Cleaning

In [ ]:
import re

def clean_non_english_chars(text):
    # Remove characters that are not typical English characters or common punctuations
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"-]', '', text)
    return cleaned_text

df = df_raw.copy()

# clean excerpt column off tags </b> and <b>
df['excerpt'] = df['excerpt'].str.replace('</b>', '').str.replace('<b>', '')

# convert date column from epoch time to human readable
df['date'] = pd.to_datetime(df['date'], unit='s').dt.strftime('%Y-%m-%d')

# apply clean_non_english_chars to article_text column
df['article_text'] = df['article_text'].apply(clean_non_english_chars)

# check if excerpt and title is in article_text, if not, concatenate them
df['article_text'] = df.apply(lambda x: x['article_text'] if x['excerpt'] in x['article_text'] else x['excerpt'] + ' ' + x['article_text'], axis=1)
df['article_text'] = df.apply(lambda x: x['article_text'] if x['title'] in x['article_text'] else x['title'] + '. ' + x['article_text'], axis=1)

# remove not found text
df['article_text'] = df['article_text'].str.replace('No article text found', '').str.replace('No body content found', '')

# remove any extra spaces and line breaks
df['article_text'] = df['article_text'].str.replace('\n', ' ').str.replace(' +', ' ')

# rename columns
df.rename(columns={'article_text': 'content'}, inplace=True)
df.rename(columns={'date': 'published_date'}, inplace=True)

# drop not needed columns
df.drop(columns=['image', 'relativeTime', 'isOld', 'syndicate', 'title', 'excerpt'], inplace=True)
df['company_name'] = company

print(df.head(5))

  published_date  ... company_name
0     2024-05-05  ...        Tesla
1     2024-05-05  ...        Tesla
2     2024-05-05  ...        Tesla
3     2024-05-05  ...        Tesla
4     2024-05-05  ...        Tesla

[5 rows x 4 columns]


Add articles to DB

In [ ]:
print(df.columns)
print(df.dtypes)

Index(['published_date', 'url', 'content', 'company_name'], dtype='object')
published_date    object
url               object
content           object
company_name      object
dtype: object


In [ ]:
schema = "hackathon_schema"
table_name = "articles"

# 4. Create the table with the specified columns
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.{table_name} (
    url STRING,
    content STRING,
    company_name STRING,
    published_date STRING
)
"""

# Execute the query
spark.sql(create_table_query)

# 5. Verify the table
verify_query = f"DESCRIBE TABLE {schema}.{table_name}"
result = spark.sql(verify_query)
result.show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|           url|   string|   NULL|
|       content|   string|   NULL|
|  company_name|   string|   NULL|
|published_date|   string|   NULL|
+--------------+---------+-------+



In [ ]:
spark_df = spark.createDataFrame(df)

# If URL is already in db, then don't add it
existing_urls = spark.sql("SELECT url FROM hackathon_schema.articles").rdd.flatMap(lambda x: x).collect()
filtered_spark_df = spark_df.filter(~spark_df.url.isin(existing_urls))

# Write data to the articles table
filtered_spark_df.write.format("delta").mode("append").saveAsTable("hackathon_schema.articles")

# Verify the insertion
result = spark.sql("SELECT * FROM hackathon_schema.articles")
result.show()


+--------------------+--------------------+------------+--------------+---------+
|                 url|             content|company_name|published_date|sentiment|
+--------------------+--------------------+------------+--------------+---------+
|https://www.msn.c...|Tesla's 'apocalyp...|       Tesla|    2024-05-05|     NULL|
|https://www.wired...|As Elon Musk Aban...|       Tesla|    2024-05-05|     NULL|
|https://www.forbe...|Longer-Range Tesl...|       Tesla|    2024-05-05|     NULL|
|https://www.msn.c...|Tesla is facing m...|       Tesla|    2024-05-05|     NULL|
|https://www.msn.c...|Tesla's Optimus v...|       Tesla|    2024-05-05|     NULL|
|https://www.msn.c...|Tesla plans to ch...|       Tesla|    2024-05-05|     NULL|
|https://news.yaho...|Hyundai antes up ...|       Tesla|    2024-05-05|     NULL|
|https://www.msn.c...|These are the kin...|       Tesla|    2024-05-05|     NULL|
|https://www.msn.c...|Musk just slashed...|       Tesla|    2024-05-04|     NULL|
|https://www.msn

In [ ]:
# Make sure we loaded all articles
result = spark.sql("SELECT DISTINCT(COUNT(*)) FROM hackathon_schema.articles")
result.show()

+--------+
|count(1)|
+--------+
|      30|
+--------+



In [ ]:
# Truncate the table
#spark.sql("TRUNCATE TABLE hackathon_schema.users")
#spark.sql("TRUNCATE TABLE hackathon_schema.articles")

DataFrame[]

In [ ]:
#spark.sql("DROP TABLE IF EXISTS hackathon_schema.articles")

DataFrame[]